#Paquetes necesarios

In [1]:
import cv2  
import math 

from ultralytics import YOLO

In [13]:
from collections import defaultdict
import numpy as np
import cv2
import math

# Carga del modelo YOLO
model1 = YOLO('yolo11n.pt')  # Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "car", "motorbike", "bus"]

# Captura desde el archivo de video
filename = "C0142.MP4"
vid = cv2.VideoCapture(filename)

# Obtenemos las propiedades del video original (ancho, alto, FPS)
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
fps = vid.get(cv2.CAP_PROP_FPS)

# Inicializamos el VideoWriter para guardar el video anotado
output_filename = 'video_anotado_solo_deteccion.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codificador de video
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

track_history = defaultdict(lambda: [])

while True:
    # fotograma a fotograma
    ret, img = vid.read()

    # Si hay imagen válida
    if ret:
        # Seguimiento con persistencia entre fotogramas
        results = model1.track(img, persist=True, classes=[0,1,2,3])

        if results is not None:
            # Creamos una imagen en negro del mismo tamaño que el fotograma original
            img_detections = np.zeros_like(img)

            # Procesamos cada detección
            for r in results:
                boxes = r.boxes

                for box in boxes:
                    # Contenedor de cada detección
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                    # Recortar el área detectada en el fotograma original
                    detection_crop = img[y1:y2, x1:x2]

                    # Pegar el área recortada en la imagen en negro
                    img_detections[y1:y2, x1:x2] = detection_crop

                    # Dibuja un recuadro y pone la etiqueta en la imagen de detecciones
                    if box.id is not None:
                        track_id = str(int(box.id[0].tolist()))
                    else:
                        track_id = ''

                    # Confianza
                    confidence = math.ceil((box.conf[0] * 100)) / 100
                    print("Confianza --->", confidence)

                    # Clase
                    cls = int(box.cls[0])
                    print("Clase -->", classNames[cls])

                    # Dibuja el contenedor y clase en la imagen de detección
                    cv2.rectangle(img_detections, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(img_detections, track_id + ' ' + classNames[cls], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Guardar el fotograma procesado en el archivo de video
            out.write(img_detections)
            cv2.imshow("Detección YOLO", img_detections)

            # Detener si se presiona la tecla "q"
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

    # Detenemos el proceso si se presiona ESC
    if cv2.waitKey(20) == 27:
        break

# Libera los recursos
vid.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 car, 83.6ms
Speed: 3.0ms preprocess, 83.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.85
Clase --> motorbike

0: 384x640 1 car, 45.0ms
Speed: 3.5ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.82
Clase --> motorbike

0: 384x640 1 car, 47.5ms
Speed: 2.5ms preprocess, 47.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.82
Clase --> motorbike

0: 384x640 1 car, 44.0ms
Speed: 1.5ms preprocess, 44.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.84
Clase --> motorbike

0: 384x640 1 car, 43.5ms
Speed: 2.0ms preprocess, 43.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.82
Clase --> motorbike

0: 384x640 1 car, 47.1ms
Speed: 2.5ms preprocess, 47.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.81
Clase --> motorbike

0: 384x640 1 car, 49.

In [11]:
from ultralytics import YOLO

# Cargar el modelo YOLOv8 (puedes usar uno preentrenado)
model = YOLO('yolov8n.pt')  # También puedes usar 'yolov8s.pt' para un modelo más grande

# Entrenar el modelo con tu dataset personalizado
model.train(
    data='license.yaml',  # El archivo de configuración YAML
    epochs=50,  # Ajusta el número de épocas según sea necesario
    imgsz=433,  # Tamaño de las imágenes (puedes ajustarlo)
    batch=16,   # Tamaño del batch
    device='cpu'    # Para entrenar con GPU, usa '0'. Para CPU, usa 'cpu'
)

New https://pypi.org/project/ultralytics/8.3.24 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.23  Python-3.9.20 torch-2.5.0+cpu CPU (AMD Ryzen 5 5600G with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=license.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=433, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

100%|██████████| 755k/755k [00:00<00:00, 4.86MB/s]

Overriding model.yaml nc=80 with nc=1



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning C:\Users\Administrator\Downloads\dataset\labels\train... 345 images, 0 backgrounds, 0 corrupt: 100%|██████████| 345/345 [00:00<00:00, 924.14it/s]


...

                   all         88         93      0.958      0.925      0.967      0.623



50 epochs completed in 0.582 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.23  Python-3.9.20 torch-2.5.0+cpu CPU (AMD Ryzen 5 5600G with Radeon Graphics)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.08s/it]


                   all         88         93      0.958      0.925      0.967      0.623
Speed: 0.6ms preprocess, 25.5ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019C6EEC6EE0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [12]:
model.val()

Ultralytics 8.3.23  Python-3.9.20 torch-2.5.0+cpu CPU (AMD Ryzen 5 5600G with Radeon Graphics)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\Administrator\Downloads\dataset\labels\val.cache... 88 images, 0 backgrounds, 0 corrupt: 100%|██████████| 88/88 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.91it/s]


                   all         88         93      0.937      0.914      0.957      0.613
Speed: 0.6ms preprocess, 23.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019C64123940>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [15]:
results = model.track(source='video_anotado_solo_deteccion.mp4', save=True, show=True)
cv2.destroyAllWindows()



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/662) c:\Users\Administrator\Documents\GitHub\Practica-4-VC\video_anotado_solo_deteccion.mp4: 256x448 (no detections), 27.0ms
video 1/1 (frame 2/662) c:\Users\Administrator\Documents\GitHub\Practica-4-VC\video_anotado_solo_deteccion.mp4: 256x448 (no detections), 24.5ms
video 1/1 (frame 3/662) c:\Users\Administrator\Documents\GitHub\Practica-4-VC\video_anotado_solo_deteccion.mp4: 256x448 (no detections), 25.0ms
video 1/1 (frame 4/662) c:

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [ ]:
# Tesseract
import cv2
import pytesseract
from pytesseract import Output

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('ocr_test.tif') 

if img is not None:
    #Convierte a RGB antes de procesar
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Texto localizado
    print(pytesseract.image_to_string(img))

    #Texto y localización en imagen de cada palabra
    d = pytesseract.image_to_data(img_rgb, output_type=Output.DICT)

    n_boxes = len(d['text'])
    for i in range(n_boxes):
        #Nivel de confianza
        if int(d['conf'][i]) > 60:
            text = d['text'][i]
            conf = d['conf'][i]
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            print(f'Texto: {text} ({conf:.2f}%)\nContenedor: {x,y,x+w,y+h}')

    cv2.imshow('img', img_rgb)
    cv2.waitKey(-1)

   

else:
    print('Error de imagen')

Reconocimiento de caracteres tras instalar easyocr

In [ ]:
import easyocr

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
res = reader.readtext('ocr_test.tif')

for (bbox, text, prob) in res:
    # Coordenadas en orden 
    (top_left, top_right, bottom_right, bottom_left) = bbox
    print(f'\nTexto: {text}\nProbabilidad: {prob:.2f}\nContenedor: {tuple(map(int, top_left)),tuple(map(int, bottom_right))}')


#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')